[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_dices/observation_prediction_dices.ipynb)

# Transduction from observation to prediction for dices

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [0]:
# Preparation

# install tensor2tensor
! pip install -U tensor2tensor

# get the needed resources
! curl -o observation_prediction_dices.sh http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.sh
! curl -o observation_prediction_dices.py http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.py
! curl -o __init__.py http://josd.github.io/eye/transduction/transduction_dices/__init__.py
! curl -o sample_dices.observation http://josd.github.io/eye/transduction/transduction_dices/sample_dices.observation
! chmod +x observation_prediction_dices.sh

# clear data and model
% rm -fr ~/t2t_data/observation_prediction_dices
% rm -fr ~/t2t_train/observation_prediction_dices/transformer-transformer_small/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir ~/t2t_train/observation_prediction_dices/transformer-transformer_small --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.2)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.0.2)
Requirement not upgraded as not directly required: gym in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.10.5)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.14.3)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (19.8.1)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2

In [0]:
# See the observation_prediction_dices problem

! pygmentize -g observation_prediction_dices.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionDices(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for dices."""

  @property
  def approx_vocab_size(self):
    return 2**4  # ~16

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 5,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 5,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      outcome = random.randint(1, 6)
      yield {
      

In [0]:
# See the observation_prediction_dices script

! pygmentize -g observation_prediction_dices.sh

#!/bin/bash
PROBLEM=observation_prediction_dices
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=1000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_dices.observation \
  --decode_hparams="beam_size=6,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_dices.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM 

In [0]:
# Run the observation_prediction_dices script

! ./observation_prediction_dices.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_dices
INFO:tensorflow:Generating data for observation_prediction_dices.
INFO:tensorflow:Generating vocab file: /content/t2t_data/observation_prediction_dices/vocab.observation_prediction_dices.16.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 43
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 38
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 38
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 38
INFO:tensorflow:Trying min_count 750
INFO:tensorflow:Iteration 0
INFO:t

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:36:29.078880: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-15 14:36:29.079355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.01GiB
2018-05-15 14:36:29.079484: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:36:29.443533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:36:29.443600: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-15 14:36:29.443629: I tensorflow/core/common_runti

INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_38_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:37:21.716033: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:37:21.716151: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:37:21.716184: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2

INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_38_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:38:12.811468: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:38:12.811605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:38:12.811637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2

INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_38_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:39:04.476940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:39:04.477048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:39:04.477093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2

...

INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:41:38.380387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:41:38.380496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:41:38.380543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-15 14:41:38.380604: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-15 14:41:38.380831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow 

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_38_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:42:29.863976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:42:29.864100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:42:29.864133: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-15 14:42:29.864162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-15 14:42:29.864

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_38_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:43:21.007619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:43:21.007728: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:43:21.007759: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-15 14:43:21.007785: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-15 14:43:21.007

INFO:tensorflow:Transforming 'targets' with symbol_modality_38_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_38_256.top
INFO:tensorflow:Base learning rate: 0.200000
INFO:tensorflow:Trainable Variables Total size: 3699200
INFO:tensorflow:Using optimizer Adam
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:44:11.903918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-15 14:44:11.904008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-15 14:44:11.904035: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-15 14:44:11.904063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-15 14:44:11.904315: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow 

INFO:tensorflow: batch 1
INFO:tensorflow:Decoding batch 0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'infer'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Beam Decoding with beam size 6
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
2018-05-15 14:45:09.053311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-15 14:45:09.053765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found d

In [0]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_dices.observation
print("->-")
! pygmentize -g sample_dices.prediction

A_THROW
A_THROW
A_THROW
A_THROW
A_THROW
A_THROW
->-
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
5	-1.68	2	-1.68	6	-1.79	3	-1.86	1	-1.88	4	-1.89
